In [2]:
import pandas as pd

In [48]:
a = '778J'
b = '41'

In [11]:
base = pd.read_csv('../data/raw/routes.txt', delimiter=',')

In [50]:
base[base['route_id'] == a + '-' + b ]['route_color'].iloc[0], type(base[base['route_id'] == a + '-' + b ]['route_color'].iloc[0])

('FF671F', str)